# Competition: https://trojandetection.ai/tracks
# Track:
#### Trojan Detection Track
In this track, we ask you to build a detector that can tell whether a given neural network contains a Trojan. We provide a dataset of networks with labels (Trojan, clean) for building your detector. Training additional networks to augment this dataset is not allowed. You will submit predictions on a set of validation/test networks to the evaluation server. The validation and test sets have held-out labels, and the evaluation server only accepts 5 submissions per day for the validation set and 5 submissions total for the test set. To participate in the Detection Track, see the following CodaLab page.

Data: The training, validation, and test sets have 1,000 networks each. Networks are split evenly across all four data sources. Half of the networks are Trojaned, and there is a 50/50 split between patch and whole-image attacks.

Metrics: Submissions will primarily be evaluated using area under the receiver operating characteristic curve (AUROC). An AUROC of 50% is random performance, and higher is better. Secondary metrics include area under the precision-recall curve (AUPR) and false-positive rate at 95% recall (FPR95). AUPR will be used to break ties. 

CodaLab: https://codalab.lisn.upsaclay.fr/competitions/5951

#### Mount Drive, Install Requirements

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# !pip install ray

In [ ]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/Trojan Detection Challenge/tdc-starter-kit/requirements.txt"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.4 MB 8.7 MB/s 
     |████████████████████████████████| 14.5 MB 21.9 MB/s 
     |████████████████████████████████| 6.8 MB 43.1 MB/s 
     |████████████████████████████████| 76 kB 4.7 MB/s 
     |████████████████████████████████| 750.6 MB 12 kB/s 
     |████████████████████████████████| 21.0 MB 1.2 MB/s 
     |████████████████████████████████| 68 kB 7.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvisio

In [ ]:
# !pip install torchensemble

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Trojan Detection Challenge/tdc-starter-kit")

##### Define Meta Neural Network

In [ ]:
import torch
import os
import json
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
cudnn.benchmark = True  # fire on all cylinders
from sklearn.metrics import roc_auc_score, roc_curve
import sys

sys.path.insert(0, '..')

## Create the dataset class

In [ ]:
class NetworkDatasetDetection(torch.utils.data.Dataset):
    def __init__(self, model_folder):
        super().__init__()
        model_paths = []
        model_paths.extend([os.path.join(model_folder, 'clean', x) \
                            for x in sorted(os.listdir(os.path.join(model_folder, 'clean')))])
        model_paths.extend([os.path.join(model_folder, 'trojan', x) \
                            for x in sorted(os.listdir(os.path.join(model_folder, 'trojan')))])
        labels = []
        data_sources = []
        for p in model_paths:
            with open(os.path.join(p, 'info.json'), 'r') as f:
                info = json.load(f)
                data_sources.append(info['dataset'])
            if p.split('/')[-2] == 'clean':
                labels.append(0)
            elif p.split('/')[-2] == 'trojan':
                labels.append(1)
            else:
                raise ValueError('unexpected path {}'.format(p))
        self.model_paths = model_paths
        self.labels = labels
        self.data_sources = data_sources
    
    def __len__(self):
        return len(self.model_paths)
    
    def __getitem__(self, index):
        return torch.load(os.path.join(self.model_paths[index], 'model.pt')), \
               self.labels[index], self.data_sources[index]

def custom_collate(batch):
    return [x[0] for x in batch], [x[1] for x in batch], [x[2] for x in batch]

## Load data
Spliting off a validation set from the train set for testing purposes.

In [ ]:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Trojan Detection Challenge'
task = 'detection'
# dataset = NetworkDatasetDetection(os.path.join(dataset_path, task, 'train'))
dataset = torch.load(os.path.join(dataset_path, "dataset.pt"))

split = int(len(dataset) * 0.8)
rnd_idx = np.random.permutation(len(dataset))
train_dataset = torch.utils.data.Subset(dataset, rnd_idx[:split])
val_dataset = torch.utils.data.Subset(dataset, rnd_idx[split:])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True,
                                           num_workers=0, pin_memory=False, collate_fn=custom_collate)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1,
                                           num_workers=0, pin_memory=False, collate_fn=custom_collate)

## Construct the MNTD network

In [ ]:
data_sources = ['CIFAR-10', 'CIFAR-100', 'GTSRB', 'MNIST']
data_source_to_channel = {k: 1 if k == 'MNIST' else 3 for k in data_sources}
data_source_to_resolution = {k: 28 if k == 'MNIST' else 32 for k in data_sources}
data_source_to_num_classes = {'CIFAR-10': 10, 'CIFAR-100': 100, 'GTSRB': 43, 'MNIST': 10}

class MetaNetwork(nn.Module):
    def __init__(self, num_queries=10, num_classes=1):
        super().__init__()
        self.queries = nn.ParameterDict(
            {k: nn.Parameter(torch.rand(num_queries,
                                        data_source_to_channel[k],
                                        data_source_to_resolution[k],
                                        data_source_to_resolution[k])) for k in data_sources}
        )
        self.affines = nn.ModuleDict(
            {k: nn.Linear(data_source_to_num_classes[k]*num_queries, 20) for k in data_sources}
        )
        # self.norm = nn.LayerNorm(20)
        self.relu = nn.ReLU(True)
        self.final_output = nn.Linear(20, num_classes)
    
    # def forward(self, data):
    #   (net, label, data_source) = data
    #   net = net[0]
    #   label = label[0]
    #   data_source = data_source[0]
    #   net.cuda().eval()

    #   return self.__forward(net, data_source)

    
    def forward(self, net, data_source):
        """
        :param net: an input network of one of the model_types specified at init
        :param data_source: the name of the data source
        :returns: a score for whether the network is a Trojan or not
        """
        query = self.queries[data_source]
        out = net(query)
        out = self.affines[data_source](out.view(1, -1))
        # out = self.norm(out)
        out = self.relu(out)
        return self.final_output(out)

In [ ]:
# from torchensemble.utils.logging import set_logger

# logger = set_logger('classification_mnist_mlp')

In [ ]:
# from torchensemble import VotingClassifier

# model = VotingClassifier(
#     estimator=MetaNetwork,
#     n_estimators=2,
#     cuda=True,
# )

In [ ]:
# num_epochs = 15
# lr = 0.01
# weight_decay = 0.
# optimizer = torch.optim.Adam(MetaNetwork().parameters(), lr=lr, weight_decay=weight_decay)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs * len(train_dataset))

In [ ]:
# model.set_optimzer(optimizer)
# model.set_criterion(F.binary_cross_entropy_with_logits())

## Train the network

In [ ]:
def train_meta_net(train_dataset, model_num):
  split = int(len(train_dataset) * 0.9)
  rnd_idx = np.random.permutation(len(train_dataset))
  actual_train_dataset = torch.utils.data.Subset(train_dataset, rnd_idx[:split])
  
  train_loader = torch.utils.data.DataLoader(actual_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=0, pin_memory=False, collate_fn=custom_collate)

  meta_network = MetaNetwork(10, num_classes=1).cuda().train()
  num_epochs = 10
  lr = 1e-3
  weight_decay=0.
  optimizer = torch.optim.Adam(meta_network.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs * len(train_dataset))

  loss_ema = np.inf
  for epoch in range(num_epochs):
    
    pbar = tqdm(train_loader)
    pbar.set_description(f"Epoch {epoch + 1}")
    for i, (net, label, data_source) in enumerate(pbar):
        net = net[0]
        label = label[0]
        data_source = data_source[0]
        net.cuda().eval()
        # net.eval()
        
        out = meta_network(net, data_source)
        
        loss = F.binary_cross_entropy_with_logits(out, torch.FloatTensor([label]).unsqueeze(0).cuda())
        # loss = F.binary_cross_entropy_with_logits(out, torch.FloatTensor([label]).unsqueeze(0))
        
        optimizer.zero_grad()
        loss.backward(inputs=list(meta_network.parameters()))
        optimizer.step()
        scheduler.step()
        for k in meta_network.queries.keys():
            meta_network.queries[k].data = meta_network.queries[k].data.clamp(0, 1)
        loss_ema = loss.item() if loss_ema == np.inf else 0.95 * loss_ema + 0.05 * loss.item()

        pbar.set_postfix(loss=loss_ema)
  if not os.path.exists('ensemble4'):
    os.makedirs('ensemble4')
  torch.save(meta_network, f'./ensemble4/{model_num}.pt')
  return meta_network


In [ ]:
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
  meta_network = train_meta_net(train_dataset, i)

  0%|          | 0/720 [00:00<?, ?it/s]

  0%|          | 0/720 [00:00<?, ?it/s]

  0%|          | 0/720 [00:00<?, ?it/s]

  0%|          | 0/720 [00:00<?, ?it/s]

  0%|          | 0/720 [00:00<?, ?it/s]

...

  0%|          | 0/720 [00:00<?, ?it/s]

  0%|          | 0/720 [00:00<?, ?it/s]

  0%|          | 0/720 [00:00<?, ?it/s]

  0%|          | 0/720 [00:00<?, ?it/s]

  0%|          | 0/720 [00:00<?, ?it/s]

In [ ]:
# meta_net_svm = torch.load('/content/drive/MyDrive/Colab Notebooks/Trojan Detection Challenge/meta_net_actual_try_2.pt')
# pbar = tqdm(train_loader)

# activation = {}
# def get_activation(name):
#     def hook(model, input, output):
#         activation[name] = output.detach()
#     return hook

# outputs = []
# labels = []
# for i, (net, label, data_source) in enumerate(pbar):
#   net = net[0]
#   label = label[0]
#   data_source = data_source[0]

#   labels.append(label)
#   net.cuda().eval()

#   meta_net_svm.affines[data_source].register_forward_hook(get_activation('affines'))
#   out = meta_net_svm(net, data_source)
#   outputs.append(activation['affines'])

# outputs[0]

In [ ]:
# # meta_net_2 = torch.load('./ensemble/1.pt')
# inp_image = meta_net_2.queries['CIFAR-10'][1:2]

# model_1 = None
# data_source_1 = None
# label_1 = None
# for p in train_loader:
#   model_1 = p[0][0]
#   data_source_1 = p[2][0]
#   label_1 = p[1][0]

#   break


In [ ]:

# inp_image = meta_net_2.queries[data_source_1][1:2]
# model_1.cuda().eval()
# out_1 = model_1(inp_image)

# out_1.shape

In [ ]:


# meta_net_2.relu.register_forward_hook(get_activation('relu'))
# meta_net_2(model_1, data_source_1)
# activation['relu']

In [ ]:
# meta_network = MetaNetwork(10, num_classes=1).cuda().train()
# # meta_network = MetaNetwork(10, num_classes=1).train()


# num_epochs = 15
# # lr = 0.019505306459592444
# # lr = 0.005
# lr = 0.01
# weight_decay = 0.
# optimizer = torch.optim.Adam(meta_network.parameters(), lr=lr, weight_decay=weight_decay)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs * len(train_dataset))

# loss_ema = np.inf
# for epoch in range(num_epochs):
    
#     pbar = tqdm(train_loader)
#     pbar.set_description(f"Epoch {epoch + 1}")
#     for i, (net, label, data_source) in enumerate(pbar):
#         net = net[0]
#         label = label[0]
#         data_source = data_source[0]
#         net.cuda().eval()
#         # net.eval()
        
#         out = meta_network(net, data_source)
        
#         loss = F.binary_cross_entropy_with_logits(out, torch.FloatTensor([label]).unsqueeze(0).cuda())
#         # loss = F.binary_cross_entropy_with_logits(out, torch.FloatTensor([label]).unsqueeze(0))
        
#         optimizer.zero_grad()
#         loss.backward(inputs=list(meta_network.parameters()))
#         optimizer.step()
#         scheduler.step()
#         for k in meta_network.queries.keys():
#             meta_network.queries[k].data = meta_network.queries[k].data.clamp(0, 1)
#         loss_ema = loss.item() if loss_ema == np.inf else 0.95 * loss_ema + 0.05 * loss.item()

#         pbar.set_postfix(loss=loss_ema)

In [ ]:
# meta_network = torch.load("/content/drive/MyDrive/Colab Notebooks/Trojan Detection Challenge/meta_net_actual_try_3.pt")

In [ ]:
# [i for i in meta_net.parameters()]

In [ ]:
# torch.save(meta_network, "/content/drive/MyDrive/Colab Notebooks/Trojan Detection Challenge/meta_net_actual_try_3.pt")

In [ ]:
# meta_network.eval()

## Evaluate the network

In [ ]:
def evaluate(meta_network, loader):
    loss_list = []
    correct_list = []
    confusion_matrix = torch.zeros(2,2)
    all_scores = []
    all_labels = []
    
    for i, (net, label, data_source) in enumerate(tqdm(loader)):
        net[0].cuda().eval()
        # net[0].eval()
        with torch.no_grad():
            out = meta_network(net[0], data_source[0])
        loss = F.binary_cross_entropy_with_logits(out, torch.FloatTensor([label[0]]).unsqueeze(0).cuda())
        # loss = F.binary_cross_entropy_with_logits(out, torch.FloatTensor([label[0]]).unsqueeze(0))
        correct = int((out.squeeze() > 0).int().item() == label[0])
        loss_list.append(loss.item())
        correct_list.append(correct)
        confusion_matrix[(out.squeeze() > 0).int().item(), label[0]] += 1
        all_scores.append(out.squeeze().item())
        all_labels.append(label[0])
    
    return np.mean(loss_list), np.mean(correct_list), confusion_matrix, all_labels, all_scores

In [ ]:
from collections import defaultdict
def get_all_scores(loader):
    all_scores = defaultdict(lambda: [])
    all_labels = []
    
    for i, (net, label, data_source) in enumerate(tqdm(loader)):
        net[0].cuda().eval()
        # net[0].eval()
        for j in range(1, 11):
          meta_network = torch.load(f'./ensemble3/{j}.pt')
          with torch.no_grad():
            out = meta_network(net[0], data_source[0])
          all_scores[j].append(out.squeeze().item())
        all_labels.append(label[0])
    
    return all_scores, all_labels

def test_get_all_scores(loader):
    all_scores = defaultdict(lambda: [])
    
    for i, (net, data_source) in enumerate(tqdm(loader)):
        net[0].cuda().eval()
        # net[0].eval()
        for j in range(1, 11):
          meta_network = torch.load(f'./ensemble3/{j}.pt')
          with torch.no_grad():
            out = meta_network(net[0], data_source[0])
          all_scores[j].append(out.squeeze().item())
    return all_scores


def get_ensembled_score(all_scores, n):
    scores = np.zeros(shape=(n))
    for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
      scores += all_scores[i]
    return scores / n

def evaluate_ensemble(scores, labels):
    correct_list = []
    confusion_matrix = torch.zeros(2,2)
    all_scores = []
    all_labels = []

    for i, score in enumerate(scores):
      correct = int(int(score > 0) == labels[i])
      correct_list.append(correct)
      confusion_matrix[int(score > 0), labels[i]] += 1
    
    return np.mean(correct_list), confusion_matrix

In [ ]:
all_scores, all_labels = get_all_scores(train_loader)
train_scores = get_ensembled_score(all_scores, len(train_dataset))
acc, cmat = evaluate_ensemble(train_scores, all_labels)
print(f'Train accuracy: {acc *100:.2f}')
print('Confusion Matrix:\n', cmat.numpy())
print(f'Train AUROC: {roc_auc_score(all_labels, train_scores):.3f}')

  0%|          | 0/800 [00:00<?, ?it/s]

Train accuracy: 92.75
Confusion Matrix:
 [[379.  38.]
 [ 20. 363.]]
Train AUROC: 0.961


In [ ]:
train_scores = get_ensembled_score(all_scores, len(train_dataset))
acc, cmat = evaluate_ensemble(train_scores, all_labels)
print(f'Train accuracy: {acc *100:.2f}')
print('Confusion Matrix:\n', cmat.numpy())
print(f'Train AUROC: {roc_auc_score(all_labels, train_scores):.3f}')

Train accuracy: 92.75
Confusion Matrix:
 [[379.  38.]
 [ 20. 363.]]
Train AUROC: 0.961


In [ ]:
all_scores, all_labels = get_all_scores(val_loader)
val_scores = get_ensembled_score(all_scores, len(val_dataset))
acc, cmat = evaluate_ensemble(val_scores, all_labels)
print(f'Validation accuracy: {acc *100:.2f}')
print('Confusion Matrix:\n', cmat.numpy())
print(f'Val AUROC: {roc_auc_score(all_labels, val_scores):.3f}')

  0%|          | 0/200 [00:00<?, ?it/s]

Validation accuracy: 91.00
Confusion Matrix:
 [[96. 13.]
 [ 5. 86.]]
Val AUROC: 0.956


In [ ]:
# train_scores = {}
# for i in range(1, 11):
#   meta_network = torch.load(f'./ensemble/{i}.pt')
#   loss, acc, cmat, all_labels, all_scores = evaluate(meta_network, train_loader)
#   train_scores[i] = {
#       'loss': loss,
#       'acc': acc,
#       'all_scores': all_scores,
#       'all_labels': all_labels
#   }
#   print(f'Train Loss: {loss:.3f}, Train Accuracy: {acc*100:.2f}')
#   print('Confusion Matrix:\n', cmat.numpy())


In [ ]:
# acc, cmat, all_labels, all_scores = evaluate_ensemble(scores/10, train_loader)
# print(f'Train accuracy: {acc *100:.2f}')
# print('Confusion Matrix:\n', cmat.numpy())
# print(f'Train AUROC: {roc_auc_score(all_labels, all_scores):.3f}')

In [ ]:
# scores = np.zeros(shape=(len(train_dataset)))
# for i in range(1, 11):
#   meta_network = torch.load(f'./ensemble/{i}.pt')
#   loss, acc, cmat, all_labels, all_scores = evaluate(meta_network, train_loader)
#   scores = scores + all_scores
#   print(f'Train Loss: {loss:.3f}, Train Accuracy: {acc*100:.2f}')
#   print('Confusion Matrix:\n', cmat.numpy())
  

# loss, acc, cmat, all_labels, all_preds = evaluate(meta_network, val_loader)
# print(f'Val Loss: {loss:.3f}, Val Accuracy: {acc*100:.2f}')
# print('Confusion Matrix:\n', cmat.numpy())

In [ ]:
# print(f'Val AUROC: {roc_auc_score(all_labels, all_preds):.3f}')

## Make submission

In [ ]:
# Dataset class for the validation/test sets, which contain all networks in a single folder

class NetworkDatasetDetectionTest(torch.utils.data.Dataset):
    def __init__(self, model_folder):
        super().__init__()
        model_paths = [os.path.join(model_folder, x) for x in sorted(os.listdir(os.path.join(model_folder)))]
        data_sources = []
        for model_path in model_paths:
            with open(os.path.join(model_path, 'info.json'), 'r') as f:
                info = json.load(f)
                data_sources.append(info['dataset'])
        self.model_paths = model_paths
        self.data_sources = data_sources
    
    def __len__(self):
        return len(self.model_paths)
    
    def __getitem__(self, index):
        return torch.load(os.path.join(self.model_paths[index], 'model.pt')), self.data_sources[index]

def custom_collate(batch):
    return [x[0] for x in batch], [x[1] for x in batch]

In [ ]:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Trojan Detection Challenge'
task = 'detection'

# test_dataset = NetworkDatasetDetectionTest(os.path.join(dataset_path, task, 'val'))
test_dataset = torch.load(os.path.join(dataset_path, 'test_dataset.pt'))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False,
                                          num_workers=0, pin_memory=False, collate_fn=custom_collate)

# dataset_path = '../../tdc_datasets'
# task = 'detection'

# test_dataset = NetworkDatasetDetectionTest(os.path.join(dataset_path, task, 'val'))
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False,
#                                           num_workers=0, pin_memory=False, collate_fn=custom_collate)

In [ ]:
def predict(meta_network, loader):
    
    all_scores = []
    for i, (net, data_source) in enumerate(tqdm(loader)):
        net[0].cuda().eval()
        # net[0].eval()
        with torch.no_grad():
            out = meta_network(net[0], data_source[0])
        all_scores.append(out.squeeze().item())
    
    return all_scores

In [ ]:
# scores = predict(meta_network, test_loader)

In [ ]:
all_scores = test_get_all_scores(test_loader)
scores = get_ensembled_score(all_scores, len(test_dataset))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
if not os.path.exists('mntd_submission_ensemble_try_3'):
    os.makedirs('mntd_submission_ensemble_try_3')

with open(os.path.join('mntd_submission_ensemble_try_3', 'predictions.npy'), 'wb') as f:
    np.save(f, np.array(scores))

!cd mntd_submission_ensemble_try_3 && zip ../mntd_submission_ensemble_try_3.zip ./* && cd ..

  adding: predictions.npy (deflated 23%)


In [ ]:
!ls

data
detection
download_datasets.py
ensemble
ensemble2
evasive_trojans
LICENSE
logs
mntd_submission
mntd_submission_1_weight_decay_early_stopped
mntd_submission_1_weight_decay_early_stopped.zip
mntd_submission_actual
mntd_submission_actual_try_2
mntd_submission_actual_try_2.zip
mntd_submission_actual.zip
mntd_submission_baseline
mntd_submission_baseline.zip
mntd_submission_ensemble_1
mntd_submission_ensemble_1.zip
mntd_submission.zip
__pycache__
README.md
requirements.txt
target_label_prediction
trigger_synthesis
utils.py
wrn.py
